In [1]:
import numpy as np
import pandas as pd
from scapy.all import *
import random, glob, os, sys

import tensorflow as tf
import keras

/Users/michaelwrana/opt/anaconda3/lib/python3.9/site-packages/scapy/layers/ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/Users/michaelwrana/opt/anaconda3/lib/python3.9/site-packages/scapy/layers/ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,
2023-02-22 12:21:09.599800: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
folder = "/Users/michaelwrana/Desktop/data/"

x = []
y = []
for filename in glob.iglob(folder + '**', recursive=True):
    if os.path.isfile(filename):
        
        # figure out what the label should be based on directory location
        # e.g. data/file1/website1 means file 1 downloaded from website 1
        file_path = filename.split('/')
        site_id = int(file_path[-2][-1])
        file_id = int(file_path[-3][-1])
        
        if(site_id == 1):
            continue
        
        # load packets from file
        packets = rdpcap(filename)
        
        # configure sequences
        seq_per_file = 100
        seq_len = 5
        packet_sequences = [0] * seq_per_file
        
        # select sequences from packets randomly
        for i in range(seq_per_file):
            start = random.randint(0,len(packets)-seq_len)
            packet_sequence = [raw(packets[j]) for j in range(start, start + seq_len)]
            packet_sequences[i] = [item for sub_list in packet_sequence for item in sub_list]
        
        # assign class labels (based on website ID or file ID)
        file_y = [file_id] * seq_per_file
        # file_y = [site_id] * seq_per_file
        
        # append to final dataset
        x = x + packet_sequences
        y = y + file_y
        
        
# pad sequences and convert to numpy array
x = tf.keras.preprocessing.sequence.pad_sequences(x, maxlen=1000)
x = np.asarray(x)
y = np.asarray(y)

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2023-02-22 12:21:19.572637: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [5]:
model.fit(x, y, epochs=45)

Epoch 1/45
625/625 [==============================] - 1s 1ms/step - loss: 5.9817 - accuracy: 0.5206
Epoch 2/45
625/625 [==============================] - 1s 989us/step - loss: 0.7085 - accuracy: 0.5238
Epoch 3/45
625/625 [==============================] - 1s 991us/step - loss: 0.6964 - accuracy: 0.5427
Epoch 4/45
625/625 [==============================] - 1s 991us/step - loss: 0.6899 - accuracy: 0.5522
Epoch 5/45
625/625 [==============================] - 1s 994us/step - loss: 0.6993 - accuracy: 0.5517
Epoch 6/45
625/625 [==============================] - 1s 996us/step - loss: 0.7109 - accuracy: 0.5522
Epoch 7/45
625/625 [==============================] - 1s 994us/step - loss: 0.6857 - accuracy: 0.5727
Epoch 8/45
625/625 [==============================] - 1s 991us/step - loss: 0.6788 - accuracy: 0.5717
Epoch 9/45
625/625 [==============================] - 1s 997us/step - loss: 0.6633 - accuracy: 0.5829
Epoch 10/45
625/625 [==============================] - 1s 997us/step - loss: 0.6660 